In [2]:
import openpyxl
import pandas as pd
import numpy as np
from numpy.linalg import norm

feature_names = ['ph',	'Hardness',	'Solids',	'Chloramines',	'Sulfate',	'Conductivity',	'Organic_carbon',	'Trihalomethanes',	'Turbidity']
noOfIterations = 20

def convertSign(value):
    if value == "positive":
        return 1
    elif value == "negative":
        return 0
    else:
        return -1

def generateSigns(feature_names):
    dataframe = openpyxl.load_workbook("outputWaterKernelShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_KernelShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_KernelShap = pd.DataFrame(dict)
     
    df_KernelShap.to_excel("signsWaterKernelShap.xlsx")
   
    #################################################################
    dataframe = openpyxl.load_workbook("outputWaterTreeShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_TreeShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3          
          
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_TreeShap = pd.DataFrame(dict)
    
    df_TreeShap.to_excel("signsWaterTreeShap.xlsx")    

    ############################################################################
    dataframe = openpyxl.load_workbook("outputWaterLIMELOOCV_20.xlsx")
    sheet = dataframe.active
    
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_LIME = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 10
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[5].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 5            
            
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_LIME = pd.DataFrame(dict)
   
    df_LIME.to_excel("signsWaterLIME.xlsx")
   

def stackRowsColumns(M, noOfRows, noOfColumns):
    pos = [0] * noOfRows
    for i in range (0, noOfRows):
        pos[i] = i
    
    for i in range(1, noOfRows):
        wsim = [0] * noOfRows
        for j in range(i, noOfRows):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
        aux = pos[max_index]
        pos[max_index] = pos[i]
        pos[i] = aux
    
    for i in range(1, noOfColumns):
        wsim = [0] * noOfColumns
        for j in range(i, noOfColumns):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
    return pos, M

def cosineSimilarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

def similarityCase(M, i):
    s = 0
    k = i - 1
    for j in range (1, k + 1):
        s = s + cosineSimilarity(M.loc[i], M.loc[i - j]) * (1 / j)
    return s

def similarityMatrix(M, noOfRows):
    s = 0
    for i in range(1, noOfRows):
        s = s + similarityCase(M, i)
    s = s / (noOfRows - 1)
    return s

def GlobalAlign(MSP, MS, noOfRows):
    # if similarityMatrix(MSP, noOfRows) >  similarityMatrix(MS, noOfRows):
    #     max = similarityMatrix(MSP, noOfRows)
    #     min = similarityMatrix(MS, noOfRows)
    # else:
    #     max = similarityMatrix(MS, noOfRows)
    #     min = similarityMatrix(MSP, noOfRows)
    # return min / max
    return similarityMatrix(MSP, noOfRows) / similarityMatrix(MS, noOfRows)


def createMSP(MP, MS, noOfRows):
    MS_copy = MS.copy(deep = True)
    MP_copy = MP.copy(deep = True)
    posS, MS_copy = stackRowsColumns(MS_copy, noOfRows, len(feature_names))
    posP, MP_copy = stackRowsColumns(MP_copy, noOfRows, len(feature_names))
    MSP = pd.DataFrame(columns = feature_names)
    for i in range(0, noOfRows):
        MSP.loc[posP[i]] = MS_copy.loc[posS[i]]
    return GlobalAlign(MSP, MS_copy, noOfRows)

generateSigns(feature_names)
df_KernelShap = pd.read_excel('signsWaterKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
df_TreeShap = pd.read_excel('signsWaterTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_LIME = pd.read_excel('signsWaterLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]

results = pd.DataFrame(columns = ['LL', 'LS', 'SL', 'SS'])
noOfInstances = df_KernelShap.shape[0]

for instance in range(0, noOfInstances):
    result = []
    data_Shap = []
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    
    data_LIME = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    
    MP = M_LIME
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_LIME
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    print(result)
    results.loc[instance] = result


results.to_excel("resultsSignsWaterMSPMS_NoMinMax.xlsx")

[1.0, 1.0155259996103638, 0.9856529266796732, 1.0]
[1.0, 0.9837604834133363, 0.931877350954911, 1.0]
[1.0, 0.9495286328577349, 0.9735256350976768, 1.0]
[1.0, 0.9638626946690615, 0.9928042788150275, 1.0]
[1.0, 0.8768833976316301, 0.9201885998555274, 1.0]
[1.0, 0.9533430313964194, 0.9499596304488797, 1.0]
[1.0, 0.9818405048430932, 0.9859266022085461, 1.0]
[1.0, 0.9627513552932022, 0.9962783598321863, 1.0]
[1.0, 0.9849676922151643, 0.9784821768908039, 1.0]
[1.0, 0.9625488193987004, 0.9905541814960673, 1.0]
[1.0, 0.9624781762046518, 0.981217688734235, 1.0]
[1.0, 0.9672982215685237, 0.9251339060719795, 1.0]
[1.0, 1.0093474828191684, 0.9872980618928847, 1.0]
[1.0, 0.9720985561464435, 0.9878711352881288, 1.0]
[1.0, 0.9865988928089882, 0.9831076958626029, 1.0]
[1.0, 0.9924088253680404, 0.9997331549178681, 1.0]
[1.0, 0.9667828033829303, 0.9774852996852391, 1.0]
[1.0, 0.9607080549801156, 0.9839350177757087, 1.0]
[1.0, 0.8512523793307035, 0.9703449581323754, 1.0]
[1.0, 0.9415969238783579, 0.98497

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppDa

[1.0, nan, 1.0796980270367826, nan]
[1.0, 0.9896165647936594, 0.9948297086338865, 1.0]
[1.0, 1.0289391430486752, 0.9746012423261271, 1.0]
[1.0, 0.918669615128162, 0.9372376459457195, 1.0]
[1.0, 0.9900558859114612, 1.0028590107701965, 1.0]
[1.0, 0.9717299529420115, 0.9992554501172991, 1.0]
[1.0, 0.9700725892652886, 0.9955601630663539, 1.0]
[1.0, 0.9209707064929912, 0.9827294715734708, 1.0]
[1.0, 0.977324314600532, 0.9869704250533392, 1.0]
[1.0, 0.973673826218732, 0.9882344237791971, 1.0]
[1.0, 0.9448822713335575, 0.968624460440958, 1.0]
[1.0, 0.9733921272880883, 0.9906312874857394, 1.0]
[1.0, 0.9826924271672997, 0.9940652940334322, 1.0]
[1.0, 0.965059227207286, 0.9334405742482196, 1.0]
[1.0, 0.9684579778597457, 0.9821252449324198, 1.0]
[1.0, 1.0, 0.9756897533239653, 1.0]
[1.0, 0.9552494926782719, 0.97776968146212, 1.0]
[1.0, 0.9249676451454866, 0.9977793253214254, 1.0]
[1.0, 0.878197888201711, 0.9901967740276295, 1.0]
[1.0, 0.9791159778912082, 0.9886406326790744, 1.0]
[1.0, 0.9899207471

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppDa

[1.0, nan, 0.9986132613037921, nan]
[1.0, 0.9570458773680134, 0.9570301771939115, 1.0]
[1.0, 0.9461799916123954, 0.9652143963999671, 1.0]
[1.0, 0.9917602595985748, 0.9690699835246157, 1.0]
[1.0, 0.9799144780691136, 0.9877727250556622, 1.0]
[1.0, 0.9981568047129367, 0.9907882844549077, 1.0]
[1.0, 1.0276892794164192, 0.9567339683557066, 1.0]
[1.0, 0.9384334152398, 0.9872819800429452, 1.0]
[1.0, 0.9393130277635501, 0.9914128095384365, 1.0]
[1.0, 0.958061716723259, 1.0038540405787544, 1.0]
[1.0, 0.9979431919315164, 0.9924585199859447, 1.0]
[1.0, 0.9908236834639269, 0.9794956872513275, 1.0]
[1.0, 0.9627653673663245, 0.9554766680764742, 1.0]
[1.0, 1.0620735185724308, 0.9893321245084389, 1.0]
[1.0, 0.9431670359175797, 0.9838240336408268, 1.0]
[1.0, 0.982764592524789, 0.9740873269220178, 1.0]
[1.0, 0.941051339091317, 0.9839400705764128, 1.0]
[1.0, 0.9607827193058627, 0.9890526994625432, 1.0]
[1.0, 0.9953072908981688, 0.961180260986316, 1.0]
[1.0, 1.0099838228990088, 0.9901143823226904, 1.0]
[1

...

[1.0, nan, 0.878748542464554, nan]
[1.0, 0.9894914193384565, 0.9140311367148763, 1.0]
[1.0, 0.9803691073048743, 0.985589091130166, 1.0]
[1.0, 0.9862068042300483, 0.9861605830208582, 1.0]
[1.0, 1.050658793749452, 0.9948355463921602, 1.0]
[1.0, 0.9227755948788083, 0.9900575173039898, 1.0]
[1.0, 1.0049748079239844, 0.9918571435940637, 1.0]
[1.0, 0.9944900372352101, 0.9909829260678207, 1.0]
[1.0, 0.9392372181780728, 0.9839403310685193, 1.0]
[1.0, 0.9145276788872957, 0.9917910789031427, 1.0]
[1.0, 0.9390048828121579, 0.9727818601704219, 1.0]
[1.0, 0.9634180983112128, 0.9642039319591408, 1.0]
[1.0, 0.9469285608051589, 0.9692964211483547, 1.0]
[1.0, 0.8858815205194505, 0.9867946775227782, 1.0]
[1.0, 0.9451240541108319, 0.9585183473389212, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppDa

[nan, 0.9801843759657706, nan, 1.0]
[1.0, 0.9994484565997821, 0.9776098144154586, 1.0]
[1.0, 0.983070642747462, 0.9850129821189905, 1.0]
[1.0, 0.9758396784925722, 0.9583955270295977, 1.0]
[1.0, 0.9865667242868934, 0.9852925300336584, 1.0]
[1.0, 0.9855476809187366, 0.9833112768531385, 1.0]
[1.0, 0.9569659546577103, 0.9317513478733557, 1.0]
[1.0, 0.9622938566580631, 0.9986601871710957, 1.0]
[1.0, 1.001874453883232, 0.999843858719086, 1.0]
[1.0, 0.9660968554142184, 0.9681454200174717, 1.0]
[1.0, 1.0171941998727283, 1.0019503351448888, 1.0]
[1.0, 0.936852785772326, 0.9785721404904562, 1.0]
[1.0, 0.986130075671665, 0.9446589676977017, 1.0]
[1.0, 0.9583050314221009, 0.9709313483629436, 1.0]
[1.0, 0.9216538060000102, 0.9541079325849727, 1.0]
[1.0, 0.9959729691993364, 0.9628327141370222, 1.0]
[1.0, 0.9516512097013555, 0.9899938012783766, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_20572\469089935.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppDa

[nan, nan, nan, nan]
[1.0, 0.9661757596781302, 0.9779677513074574, 1.0]
[1.0, 0.9941490899195078, 0.9948383729553744, 1.0]
[1.0, 0.942992856414905, 0.9758774724765975, 1.0]
[1.0, 0.9730289889372631, 0.9833373447025755, 1.0]
[1.0, 0.9828627373953064, 0.9593604388415105, 1.0]
[1.0, 0.9319316197066967, 0.9924911001890958, 1.0]
[1.0, 0.9561379472730255, 0.985260887921826, 1.0]
[1.0, 0.9463134806143972, 0.9673150248551713, 1.0]
[1.0, 0.9319428411695568, 0.9736514720851869, 1.0]
[1.0, 1.0037203042849405, 0.9815048790572355, 1.0]
[1.0, 1.0105924245222848, 1.000276273011485, 1.0]
[1.0, 1.0052248196465758, 0.9889110600162538, 1.0]
[1.0, 0.9693983937197349, 0.9902289992487013, 1.0]
[1.0, 0.9799880778215124, 0.9599849838386876, 1.0]
[1.0, 0.8994519409075942, 0.9696550461362211, 1.0]
[1.0, 0.9827755250567853, 0.9843788338985157, 1.0]
[1.0, 0.9924394164142376, 0.9582909651688917, 1.0]
[1.0, 0.9984583909535808, 0.9610076025115735, 1.0]
[1.0, 0.9835929920720469, 0.992802335343778, 1.0]
[1.0, 0.891043